In [2]:
## 1. Motivation
The 2021-2022 Boulder County fires (Marshall Fire) is reported on 30 December 2021 and ended on January 1, 2022. The wildfire has destroyed 1,084 Buildings and burned 6,026 acres. The damage and lost for our Boulder community, accompanied by Middle Fork Fire, demand our attention.

The general motivatio
n of building data model for this project is to do exploratory data analysis on the history of wildfires in the U.S., see how they’ve changed over time and build a data model to predict the cause of a wildfire based on location, time, and etc.

SyntaxError: invalid syntax (3095011831.py, line 2)

In [4]:
# Load required library for project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import sqlite3



## 2. Data Resource and Explanation of Variables
The dataset used for this report was found via the US Department of Agriculture. It provides information on 2,166,753 wildfires in the U.S. from 1992-2018, with a variety of information including spacial, cause, size, discovery/containment dates and different classifications.  

Thanks to [**U.S. DEPARTMENT OF AGRICULTURE**](https://www.fs.usda.gov/rds/archive/Catalog/RDS-2013-0009.5) for providing the dataset.

The data for this project was obtained via the websites mentioned above and compiled into one sqlite file to be read into jupiter notebook.

The description for each variable inside the dataset can be found a dataset published on the [**Kaggle** dataset website](https://www.kaggle.com/rtatman/188-million-us-wildfires). This website provides the yearly wildfire data for the United States.

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("FPA_FOD_20210617.sqlite")
df = pd.read_sql_query("SELECT NWCG_GENERAL_CAUSE, FIRE_SIZE, DISCOVERY_DATE, STATE, "
                       "FIPS_CODE, FIPS_NAME, LATITUDE, LONGITUDE, "
                       "FIRE_SIZE_CLASS, CONT_DATE, FIRE_YEAR, FIRE_NAME, "
                       "NWCG_REPORTING_UNIT_NAME FROM Fires", con)
con.close()
print(df.info())

In [ ]:
# Filter useless cause
df = df[df['NWCG_GENERAL_CAUSE'] != 'Missing data/not specified/undetermined']
df = df[df['NWCG_GENERAL_CAUSE'] != 'Other causes']

In [ ]:
df.head()

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Monthly
import pandas as pd

In [ ]:
df_test = df

In [ ]:
df_test['DISCOVERY_DATE'] = pd.to_datetime(df_test['DISCOVERY_DATE'])
df_test['month'] = df_test['DISCOVERY_DATE'].dt.month
df_test['day'] = df_test['DISCOVERY_DATE'].dt.day
df_test['year'] = df_test['DISCOVERY_DATE'].dt.year


In [ ]:
df_test.info()

In [ ]:
def getweather(s): 
    month = s['month']
    day = s['day']
    year = s['year']
    lat = s['LATITUDE']
    lon = s['LONGITUDE']
    
    start = datetime(year, month, day)
    end = datetime(year, month, day)
    location = Point(lat, lon)
    data = Daily(location, start, end)
    data = data.fetch()
    
    if not data.empty :
        s['tavg'] = data['tavg'].values[0].tolist()
        s['tmin'] = data.tmin.values[0].tolist()
        s['tmax'] = data.tmax.values[0].tolist()
        s['prcp'] = data.prcp.values[0].tolist()
        s['snow'] = data.snow.values[0].tolist()
        s['wdir'] = data.wdir.values[0].tolist()
        s['wspd'] = data.wspd.values[0].tolist()
        s['wpgt'] = data.wpgt.values[0].tolist()
        s['pres'] = data.pres.values[0].tolist()
        s['tsun'] = data.tsun.values[0].tolist()
    return s

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

#df_test['response'] = df_test.apply(lambda x: getweather(x.month, x.day, x.year, x.LATITUDE, x.LONGITUDE), axis=1)
from tqdm import tqdm
tqdm.pandas()
pd.options.mode.chained_assignment = None

df_test = df_test.progress_apply(getweather, axis=1)

In [ ]:
df_test = df_test[df_test['tavg'].notna()]
df_test.info()

In [ ]:
df_test.to_csv('us_weather.csv', index = False)

In [ ]:
df_test.drop('wpgt', inplace=True, axis = 1)
df_test.drop('tsun', inplace=True, axis = 1)
df_test.drop('snow', inplace=True, axis = 1)
df_test.drop('wdir', inplace=True, axis = 1)

In [ ]:
df_test.info()

In [ ]:
df_ml = df_test

In [ ]:
df_ml.info()

In [ ]:
df_ml.drop('NWCG_REPORTING_UNIT_NAME', inplace=True, axis = 1)
df_ml.drop('FIRE_SIZE_CLASS', inplace=True, axis = 1)
df_ml.drop('FIRE_NAME', inplace=True, axis = 1)
df_ml.drop('FIRE_YEAR', inplace=True, axis = 1)

In [ ]:
df_ml.head()

In [ ]:
df_ml.drop('FIPS_NAME', inplace=True, axis = 1)
df_ml.drop('FIPS_CODE', inplace=True, axis = 1)

In [ ]:
df_ml.drop('CONT_DATE', inplace=True, axis = 1)
df_ml.drop('DISCOVERY_DATE', inplace=True, axis = 1)

In [ ]:
df_ml.head()

In [ ]:
df_ml = df_ml[df_ml['FIRE_SIZE'] < 100]

In [ ]:
df_ml.info()

In [ ]:
df_ml.drop('year', inplace=True, axis = 1)

In [ ]:
tem = df_ml.dropna()
tem.info()

In [ ]:
tem.to_csv('tem_weather.csv', index = False)

In [19]:
df_test = df

In [20]:
df_test['DISCOVERY_DATE'] = pd.to_datetime(df_test['DISCOVERY_DATE'])
df_test['month'] = df_test['DISCOVERY_DATE'].dt.month
df_test['day'] = df_test['DISCOVERY_DATE'].dt.day
df_test['year'] = df_test['DISCOVERY_DATE'].dt.year


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609552 entries, 0 to 2166726
Data columns (total 16 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   NWCG_GENERAL_CAUSE        1609552 non-null  object        
 1   FIRE_SIZE                 1609552 non-null  float64       
 2   DISCOVERY_DATE            1609552 non-null  datetime64[ns]
 3   STATE                     1609552 non-null  object        
 4   FIPS_CODE                 1092313 non-null  object        
 5   FIPS_NAME                 1092312 non-null  object        
 6   LATITUDE                  1609552 non-null  float64       
 7   LONGITUDE                 1609552 non-null  float64       
 8   FIRE_SIZE_CLASS           1609552 non-null  object        
 9   CONT_DATE                 1014951 non-null  object        
 10  FIRE_YEAR                 1609552 non-null  int64         
 11  FIRE_NAME                 940703 non-null   object

In [22]:
def getweather(s): 
    month = s['month']
    day = s['day']
    year = s['year']
    lat = s['LATITUDE']
    lon = s['LONGITUDE']
    
    start = datetime(year, month, day)
    end = datetime(year, month, day)
    location = Point(lat, lon)
    data = Daily(location, start, end)
    data = data.fetch()
    
    if not data.empty :
        s['tavg'] = data['tavg'].values[0].tolist()
        s['tmin'] = data.tmin.values[0].tolist()
        s['tmax'] = data.tmax.values[0].tolist()
        s['prcp'] = data.prcp.values[0].tolist()
        s['snow'] = data.snow.values[0].tolist()
        s['wdir'] = data.wdir.values[0].tolist()
        s['wspd'] = data.wspd.values[0].tolist()
        s['wpgt'] = data.wpgt.values[0].tolist()
        s['pres'] = data.pres.values[0].tolist()
        s['tsun'] = data.tsun.values[0].tolist()
    return s

In [23]:
%%time
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

#df_test['response'] = df_test.apply(lambda x: getweather(x.month, x.day, x.year, x.LATITUDE, x.LONGITUDE), axis=1)
from tqdm import tqdm
tqdm.pandas()
pd.options.mode.chained_assignment = None

df_test = df_test.progress_apply(getweather, axis=1)

100%|██████████| 1609552/1609552 [35:29:26<00:00, 12.60it/s]    

Wall time: 1d 11h 29min 26s


In [24]:
df_test = df_test[df_test['tavg'].notna()]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582976 entries, 5 to 2166726
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   CONT_DATE                 361911 non-null  object        
 1   DISCOVERY_DATE            582976 non-null  datetime64[ns]
 2   FIPS_CODE                 472186 non-null  object        
 3   FIPS_NAME                 472186 non-null  object        
 4   FIRE_NAME                 349320 non-null  object        
 5   FIRE_SIZE                 582976 non-null  float64       
 6   FIRE_SIZE_CLASS           582976 non-null  object        
 7   FIRE_YEAR                 582976 non-null  int64         
 8   LATITUDE                  582976 non-null  float64       
 9   LONGITUDE                 582976 non-null  float64       
 10  NWCG_GENERAL_CAUSE        582976 non-null  object        
 11  NWCG_REPORTING_UNIT_NAME  582976 non-null  object        
 12  S

In [30]:
df_test.to_csv('us_weather.csv', index = False)

In [26]:
df_test.drop('wpgt', inplace=True, axis = 1)
df_test.drop('tsun', inplace=True, axis = 1)
df_test.drop('snow', inplace=True, axis = 1)
df_test.drop('wdir', inplace=True, axis = 1)

In [29]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582976 entries, 5 to 2166726
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   CONT_DATE                 361911 non-null  object        
 1   DISCOVERY_DATE            582976 non-null  datetime64[ns]
 2   FIPS_CODE                 472186 non-null  object        
 3   FIPS_NAME                 472186 non-null  object        
 4   FIRE_NAME                 349320 non-null  object        
 5   FIRE_SIZE                 582976 non-null  float64       
 6   FIRE_SIZE_CLASS           582976 non-null  object        
 7   FIRE_YEAR                 582976 non-null  int64         
 8   LATITUDE                  582976 non-null  float64       
 9   LONGITUDE                 582976 non-null  float64       
 10  NWCG_GENERAL_CAUSE        582976 non-null  object        
 11  NWCG_REPORTING_UNIT_NAME  582976 non-null  object        
 12  S

In [31]:
df_ml = df_test

In [32]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582976 entries, 5 to 2166726
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   CONT_DATE                 361911 non-null  object        
 1   DISCOVERY_DATE            582976 non-null  datetime64[ns]
 2   FIPS_CODE                 472186 non-null  object        
 3   FIPS_NAME                 472186 non-null  object        
 4   FIRE_NAME                 349320 non-null  object        
 5   FIRE_SIZE                 582976 non-null  float64       
 6   FIRE_SIZE_CLASS           582976 non-null  object        
 7   FIRE_YEAR                 582976 non-null  int64         
 8   LATITUDE                  582976 non-null  float64       
 9   LONGITUDE                 582976 non-null  float64       
 10  NWCG_GENERAL_CAUSE        582976 non-null  object        
 11  NWCG_REPORTING_UNIT_NAME  582976 non-null  object        
 12  S

In [33]:
df_ml.drop('NWCG_REPORTING_UNIT_NAME', inplace=True, axis = 1)
df_ml.drop('FIRE_SIZE_CLASS', inplace=True, axis = 1)
df_ml.drop('FIRE_NAME', inplace=True, axis = 1)
df_ml.drop('FIRE_YEAR', inplace=True, axis = 1)

In [34]:
df_ml.head()

,CONT_DATE,DISCOVERY_DATE,FIPS_CODE,FIPS_NAME,FIRE_SIZE,LATITUDE,LONGITUDE,NWCG_GENERAL_CAUSE,STATE,day,month,prcp,pres,tavg,tmax,tmin,wspd,year
5,7/1/2004 0:00,2004-06-30,06005,Amador County,0.1,38.635278,-120.103611,Natural,CA,30,6,NaN,1017.2,11.7,21.7,4.4,7.9,2004
6,7/2/2004 0:00,2004-07-01,06017,El Dorado County,0.1,38.688333,-120.153333,Natural,CA,1,7,NaN,1019.2,11.9,20.0,3.3,6.6,2004
10,7/3/2004 0:00,2004-07-02,06017,El Dorado County,0.1,38.691667,-120.159722,Natural,CA,2,7,NaN,1019.4,13.5,23.3,3.3,10.2,2004
14,10/3/2004 0:00,2004-10-03,06017,El Dorado County,0.2,38.675833,-120.279722,Natural,CA,3,10,NaN,1022.8,9.1,20.6,-1.1,4.8,2004
17,10/17/2004 0:00,2004-10-13,06017,El Dorado County,7700.0,38.780000,-120.260000,Power generation/transmission/distribution,CA,13,10,NaN,1023.4,9.4,19.4,-0.6,8.6,2004


In [35]:
df_ml.drop('FIPS_NAME', inplace=True, axis = 1)
df_ml.drop('FIPS_CODE', inplace=True, axis = 1)

In [39]:
df_ml.drop('CONT_DATE', inplace=True, axis = 1)
df_ml.drop('DISCOVERY_DATE', inplace=True, axis = 1)

In [40]:
df_ml.head()

,FIRE_SIZE,LATITUDE,LONGITUDE,NWCG_GENERAL_CAUSE,STATE,day,month,prcp,pres,tavg,tmax,tmin,wspd,year
5,0.1,38.635278,-120.103611,Natural,CA,30,6,NaN,1017.2,11.7,21.7,4.4,7.9,2004
6,0.1,38.688333,-120.153333,Natural,CA,1,7,NaN,1019.2,11.9,20.0,3.3,6.6,2004
10,0.1,38.691667,-120.159722,Natural,CA,2,7,NaN,1019.4,13.5,23.3,3.3,10.2,2004
14,0.2,38.675833,-120.279722,Natural,CA,3,10,NaN,1022.8,9.1,20.6,-1.1,4.8,2004
17,7700.0,38.780000,-120.260000,Power generation/transmission/distribution,CA,13,10,NaN,1023.4,9.4,19.4,-0.6,8.6,2004


In [42]:
df_ml = df_ml[df_ml['FIRE_SIZE'] < 100]

In [44]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570785 entries, 5 to 2166698
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   FIRE_SIZE           570785 non-null  float64
 1   LATITUDE            570785 non-null  float64
 2   LONGITUDE           570785 non-null  float64
 3   NWCG_GENERAL_CAUSE  570785 non-null  object 
 4   STATE               570785 non-null  object 
 5   day                 570785 non-null  int64  
 6   month               570785 non-null  int64  
 7   prcp                373629 non-null  float64
 8   pres                401640 non-null  float64
 9   tavg                570785 non-null  float64
 10  tmax                569845 non-null  float64
 11  tmin                569878 non-null  float64
 12  wspd                557804 non-null  float64
 13  year                570785 non-null  int64  
dtypes: float64(9), int64(3), object(2)
memory usage: 65.3+ MB


In [48]:
df_ml.drop('year', inplace=True, axis = 1)

In [49]:
tem = df_ml.dropna()
tem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321577 entries, 35 to 2166661
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   FIRE_SIZE           321577 non-null  float64
 1   LATITUDE            321577 non-null  float64
 2   LONGITUDE           321577 non-null  float64
 3   NWCG_GENERAL_CAUSE  321577 non-null  object 
 4   STATE               321577 non-null  object 
 5   day                 321577 non-null  int64  
 6   month               321577 non-null  int64  
 7   prcp                321577 non-null  float64
 8   pres                321577 non-null  float64
 9   tavg                321577 non-null  float64
 10  tmax                321577 non-null  float64
 11  tmin                321577 non-null  float64
 12  wspd                321577 non-null  float64
dtypes: float64(9), int64(2), object(2)
memory usage: 34.3+ MB


In [50]:
tem.to_csv('tem_weather.csv', index = False)

In [ ]:
df = pd.read_csv('US_Wildfire.csv')

In [ ]:
df.info()

In [ ]:
df[-year]

In [5]:
df = pd.read_csv('US_Wildfire.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582976 entries, 0 to 582975
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CONT_DATE                 361911 non-null  object 
 1   DISCOVERY_DATE            582976 non-null  object 
 2   FIPS_CODE                 472186 non-null  float64
 3   FIPS_NAME                 472186 non-null  object 
 4   FIRE_NAME                 338615 non-null  object 
 5   FIRE_SIZE                 582976 non-null  float64
 6   FIRE_SIZE_CLASS           582976 non-null  object 
 7   FIRE_YEAR                 582976 non-null  int64  
 8   LATITUDE                  582976 non-null  float64
 9   LONGITUDE                 582976 non-null  float64
 10  NWCG_GENERAL_CAUSE        582976 non-null  object 
 11  NWCG_REPORTING_UNIT_NAME  582976 non-null  object 
 12  STATE                     582976 non-null  object 
 13  day                       582976 non-null  i

In [8]:
df[-year]

NameError: name 'year' is not defined